<a href="https://colab.research.google.com/github/Matrosee/Genre-Klassifikation-anhand-von-Liedtexten/blob/main/Code_von_Jannic_M_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installationen, Imports und Downloads:

Installationen:

In [ ]:
!pip install lightgbm
!pip install spacy
!pip install scikit-learn
!pip install numpy
!python -m spacy download en_core_web_md
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install flaml

Imports:

In [20]:
import flaml
import lightgbm as lgb
import nltk
import numpy as np
import pandas as pd
import re
import spacy
from flaml import AutoML
from google.colab import files
from google.colab import drive
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

Downloads:

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

# Upload & Dataframe Deklaration:

In [22]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
df = pd.read_csv('/content/gdrive/MyDrive/train.csv')

In [24]:
df_test = pd.read_csv('/content/gdrive/MyDrive/test.csv')

# 1890 Songs-pro-Datensatz erstellen:

In [25]:
# Nur Englische-Lieder einstellen
df = df.loc[df['Language']=='en']

# Pro Genre ein eigenes Dataframe mit je 1890 Liedern
df_rock_one = df[df['Genre']=='Rock']
df_rock_one = df_rock_one.head(1890)

df_pop_one = df[df['Genre']=='Pop']
df_pop_one = df_pop_one.head(1890)

df_metal_one = df[df['Genre']=='Metal']
df_metal_one = df_metal_one.head(1890)

df_hip_hop_one = df[df['Genre']=='Hip-Hop']
df_hip_hop_one = df_hip_hop_one.head(1890)

df_rnb_one = df[df['Genre']=='R&B']
df_rnb_one = df_rnb_one.head(1890)

df_indie_one = df[df['Genre']=='Indie']
df_indie_one = df_indie_one.head(1890)

df_electronic_one = df[df['Genre']=='Electronic']
df_electronic_one = df_electronic_one.head(1890)

df_jazz_one = df[df['Genre']=='Jazz']
df_jazz_one = df_jazz_one.head(1890)

df_folk_one = df[df['Genre']=='Folk']
df_folk_one = df_folk_one.head(1890)

df_country_one = df[df['Genre']=='Country']
df_country_one = df_country_one.head(1890)

# Alle einzelnen Dataframes zu einem zusammfügen
df_one = pd.concat([df_pop_one, df_hip_hop_one, df_metal_one, df_rock_one, 
                    df_indie_one, df_country_one, df_electronic_one, df_rnb_one,
                    df_jazz_one, df_folk_one])

# Kürzeren Namen für zuküftige Arbeit auswählen
df = df_one

# Spalten ohne Beiträge löschen 
df.dropna()

# Index reseten
df = df.reset_index(drop=True)

# Pre-Processing:

In [26]:
df = df.dropna(axis=1)

In [27]:
# Laden Sie das spacy-Modell und den Englisch-Vokabular
nlp = spacy.load("en_core_web_md")

# Stop-Wörter auf einer Variable deklarieren
en_stops = set(stopwords.words('english'))

Datensatz säubern:

In [28]:
# Sonderzeichen entfernen
df['Lyrics'] = [re.sub(r'^.*?Lyrics', '', str(lyric)) for lyric in df['Lyrics']]

# Satzumbruch Zeichen entfernen
df['cleaned_lyrics'] = [str(lyric).replace('\n',' ') for lyric in df['Lyrics']]

# Wörter in eckigen Klammern entfernen
df['cleaned_lyrics'] = [re.sub("\[.*?\]","",lyric) for lyric in df['cleaned_lyrics']]

# Language-, Song-Name- und Artist-Spalte löschen
df = df.drop('Language',axis=1)
df = df.drop('Song', axis=1)
df = df.drop('Artist', axis=1)

# Den Index resetten
df.reset_index(drop=True)

,Genre,Lyrics,cleaned_lyrics
0,Pop,Twenty-five years and my life is still\nTrying...,Twenty-five years and my life is still Trying ...
1,Pop,Starry night bring me down\nTill I realize the...,Starry night bring me down Till I realize the ...
2,Pop,Every time you wake in the mornin'\nAnd you st...,Every time you wake in the mornin' And you sta...
3,Pop,What ya gonna do child\nWhen your thoughts are...,What ya gonna do child When your thoughts are ...
4,Pop,"How can you tell, when your wellness is not we...","How can you tell, when your wellness is not we..."
...,...,...,...
18895,Folk,One for the money and two for the show\nThree ...,One for the money and two for the show Three f...
18896,Folk,"Carmilla are you still there-ere, I heard you ...","Carmilla are you still there-ere, I heard you ..."
18897,Folk,"Oh Jo-osephine, well you won't fi-ind your dre...","Oh Jo-osephine, well you won't fi-ind your dre..."
18898,Folk,"I've got my li-ife, you've got your li-ife\nWe...","I've got my li-ife, you've got your li-ife We ..."


Stemming:

In [29]:
# Neue Spalte erstellen und die Liedtexte ohne Endungen in die Spalte speichen
df['stemmed_lyrics'] = ""
ps = PorterStemmer() 
space = " "
tmp = ""
count = 0
for lyric in df['cleaned_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for word in words:
    tmp = tmp + space + ps.stem(word)
  df['stemmed_lyrics'].iloc[count] = tmp
  count = count + 1

Stop-Words:

In [30]:
# Neue Spalte erstellen und die Liedtexte ohne Stoppwörter in die Spalte speichen
df['wosw_lyrics'] = ""
space = " "
tmp = ""
count = 0
for lyric in df['cleaned_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for word in words:
    if word not in en_stops:
      tmp = tmp + space + word
  df['wosw_lyrics'].iloc[count] = tmp
  count = count + 1

Stemming -> Stop-Words:

In [31]:
# Neue Spalte erstellen und die Liedtexte ohne Endungen und ohne Stoppwörter in die Spalte speichen
df['stemmed_wosw_lyrics'] = ""
space = " "
tmp = ""
count = 0
for lyric in df['stemmed_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for word in words:
    if word not in en_stops:
      tmp = tmp + space + word
  df['stemmed_wosw_lyrics'].iloc[count] = tmp
  count = count + 1

StopWords -> Stemming:

In [32]:
# Neue Spalte erstellen und die Liedtexte ohne Stoppwörter und ohne Endungen in die Spalte speichen
df['wosw_stemmed_lyrics'] = ""
ps = PorterStemmer() 
space = " "
tmp = ""
count = 0
for lyric in df['wosw_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for x in words:
    tmp = tmp + space + ps.stem(x)
  df['wosw_stemmed_lyrics'].iloc[count] = tmp
  count = count + 1

# Einstellen, welche Lyrik-Spalten verwendet werden sollen

In [33]:
# Laden Sie die Textdokumente und die Klassenlabels
df_train_lyrics = df['Lyrics'].tolist() # Dataframe mit Textdokumenten
df_train_labels = df['Genre'] # Dataframe mit Klassenlabels (eins pro Dokument)

In [34]:
# Laden Sie die Textdokumente und die Klassenlabels
df_test_lyrics = df_test['Lyrics'] # Dataframe mit Textdokumenten
df_test_labels = df_test['Genre'] # Dataframe mit Klassenlabels (eins pro Dokument)

# Grid-search SVM mit SKLEARN:

Wie ich die optimalen Paramter rausfinde?
Ich gebe grobe Werte für die Paramter an und werde immer genauer

CountVektorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.2, random_state=42)

# Erstellen Sie ein SVM-Modell
svm = SVC()

# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
param_grid_svm = {'C': [0.011, 0.012, 0.014], 
                  'gamma': [0.075, 0.079, 0.0795],
                  'kernel': ['linear']}

# Erstelle eine GridSearchCV-Instanz 
grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3)

# Trainiere dein Modell mit der GridSearchCV
grid_svm.fit(X_train, y_train)

# Auswähle den besten Parameter
best_params_svm = grid_svm.best_params_

# Die besten Parameter ausgeben
print(best_params_svm)

SVM mit TF-IDF

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.2, random_state=42)

# Erstellen Sie ein SVM-Modell
svm = SVC()

# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
param_grid_svm = {'C': [4.5], 
                  'gamma': [0.95,1,3,5,10],
                  'kernel': ['rbf']}

# Erstelle eine GridSearchCV-Instanz 
grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3)

# Trainiere dein Modell mit der GridSearchCV
grid_svm.fit(X_train, y_train)

# Auswähle den besten Parameter
best_params_svm = grid_svm.best_params_

# Die besten Parameter ausgeben
print(best_params_svm)

Spacy:

In [ ]:
# # Funktion definieren, um die Liedtexte in Vektoren zu konvertieren
# def document_vector(text):
#     doc = nlp(text)
#     vectors = []
#     for token in doc:
#         if token.has_vector:
#             vectors.append(token.vector)
#     if len(vectors) > 0:
#         return np.mean(vectors, axis=0)
#     else:
#         raise ValueError("Keine Vektoren gefunden")

# # Herausfinden, in welchen Spalten keine Vektoren vorhanden sind
# no_vector_docs = []
# for i, text in enumerate(df_train_lyrics):
#     try:
#         vec = document_vector(text)
#     except ValueError as e:
#         print(f"Fehler: {e} bei Dokument {i}")
#         no_vector_docs.append(i)
# print(f"Dokumente ohne Vektoren: {no_vector_docs}")

# # Alle Zeilen ohne Vektoren löschen
# for i in no_vector_docs:
#   df_train_labels = df_train_labels.drop(i)
#   del df_train_lyrics[i]

# # Das Dataframe für die Labels reindexen
# df_train_labels = df_train_labels.reset_index(drop=True)  

# # Compute the document vectors
# documents_with_vectors = []
# labels_with_vectors = []
# for i in range(len(df_train_lyrics)):
#     text = df_train_lyricss[i]
#     df_train_labels = df_train_labels[i]
#     try:
#         vec = document_vector(text)
#         documents_with_vectors.append(text)
#         labels_with_vectors.append(label)
#     except ValueError as e:
#         print(f"Fehler: {e}")

# # Compute the document vectors
# X = [document_vector(text) for text in documents_with_vectors]

# # Teilen Sie die Daten in Trainings- und Testdaten auf
# X_train, X_test, y_train, y_test = train_test_split(X, labels_with_vectors, test_size=0.2, random_state=42)

# # Erstellen Sie ein SVM-Modell
# svm = SVC()

# # Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
# param_grid_svm = {'C': [1, 5], 
#                   'gamma': [10, 1, 0.1],
#                   'kernel': ['linear']}

# # Erstelle eine GridSearchCV-Instanz 
# grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3)

# # Trainiere dein Modell mit der GridSearchCV
# grid_svm.fit(X_train, y_train)

# # Auswähle den besten Parameter
# best_params_svm = grid_svm.best_params_

# # Die besten Parameter ausgeben
# print(best_params_svm)

# Gridsearch LGBM mit FLAML:

CountVectorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.15, random_state=42)

# Das Flaml-Modell deklarieren und trainieren
automl = AutoML()
automl.fit(X_train, y_train, task="classification", metric='accuracy', estimator_list=["lgbm"], time_budget=3600)

# Die besten Hyperparameter ausgeben
print('Best hyperparmeter config:', automl.best_config)

TfidfVectorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem CountVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.15, random_state=42)

# Das Flaml-Modell deklarieren und trainieren
automl = AutoML()
automl.fit(X_train, y_train, task="classification", metric='accuracy', estimator_list=["lgbm"], time_budget=6000)

# Die besten Hyperparameter ausgeben
print('Best hyperparmeter config:', automl.best_config)

Spacy-Vektoren:

In [ ]:
# # Funktion definieren, um die Liedtexte in Vektoren zu konvertieren
# def document_vector(text):
#     doc = nlp(text)
#     vectors = []
#     for token in doc:
#         if token.has_vector:
#             vectors.append(token.vector)
#     if len(vectors) > 0:
#         return np.mean(vectors, axis=0)
#     else:
#         raise ValueError("Keine Vektoren gefunden")

# # Herausfinden, in welchen Spalten keine Vektoren vorhanden sind
# no_vector_docs = []
# for i, text in enumerate(df_train_lyrics):
#     try:
#         vec = document_vector(text)
#     except ValueError as e:
#         print(f"Fehler: {e} bei Dokument {i}")
#         no_vector_docs.append(i)
# print(f"Dokumente ohne Vektoren: {no_vector_docs}")

# # Alle Zeilen ohne Vektoren löschen
# for i in no_vector_docs:
#   df_train_labels = df_train_labels.drop(i)
#   del df_train_lyrics[i]

# # Das Dataframe für die Labels reindexen
# df_train_labels = df_train_labels.reset_index(drop=True)  

# # Compute the document vectors
# documents_with_vectors = []
# labels_with_vectors = []
# for i in range(len(df_train_lyrics)):
#     text = df_train_lyricss[i]
#     df_train_labels = df_train_labels[i]
#     try:
#         vec = document_vector(text)
#         documents_with_vectors.append(text)
#         labels_with_vectors.append(label)
#     except ValueError as e:
#         print(f"Fehler: {e}")

# # Compute the document vectors
# X = [document_vector(text) for text in documents_with_vectors]

# # Teilen Sie die Daten in Trainings- und Testdaten auf
# X_train, X_test, y_train, y_test = train_test_split(X, labels_with_vectors, test_size=0.2, random_state=42)

# # Das Flaml-Modell deklarieren und trainieren
# automl = AutoML()
# automl.fit(np.array(X_train), np.array(y_train), task="classification", metric='accuracy', estimator_list=["lgbm"], time_budget=6000)


# # Die besten Hyperparameter ausgeben
# print('Best hyperparmeter config:', automl.best_config)

# MLA ohne Hyperparameter

CountVectorizer:

In [35]:
# Vektorizer initialisiereen
vectorizer = CountVectorizer()

# Variablen für MLA deklarieren und teils konvertieren
X_train = vectorizer.fit_transform(df_train_lyrics)
X_test = vectorizer.transform(df_test_lyrics)

y_train = df_train_labels
y_test = df_test_labels

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier()
svm = SVC()

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)

TF-IDF Vectorizer:

In [ ]:
# Vektorizer initialisiereen
vectorizer = TfidfVectorizer()

# Variablen für MLA deklarieren und teils konvertieren
X_train = vectorizer.fit_transform(df_train_lyrics)
X_test = vectorizer.transform(df_test_lyrics)

y_train = df_train_labels
y_test = df_test_labels

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier()
svm = SVC()

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)

# MLA mit Hyperparameter

CountVectorizer:

In [37]:
# Vektorizer initialisiereen
vectorizer = CountVectorizer()

# Variablen für MLA deklarieren und teils konvertieren
X_train = vectorizer.fit_transform(df_train_lyrics)
X_test = vectorizer.transform(df_test_lyrics)

y_train = df_train_labels
y_test = df_test_labels

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier(n_estimators=200, num_leaves= 184, min_child_samples=3, 
                          learning_rate=0.08838576141784195, log_max_bin=9, 
                          colsample_bytree=0.39039795455449644, 
                          reg_alpha=0.014844095616079196, 
                          reg_lambda=0.05525097930389173)
svm = SVC(C=0.01, gamma=0.07, kernel='linear')

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)

TF-IDF Vectorizer:

In [ ]:
# Vektorizer initialisiereen
vectorizer = TfidfVectorizer()

# Variablen für MLA deklarieren und teils konvertieren
X_train = vectorizer.fit_transform(df_train_lyrics)
X_test = vectorizer.transform(df_test_lyrics)

y_train = df_train_labels
y_test = df_test_labels

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier(n_estimators=67, num_leaves= 184, min_child_samples=12, 
                          learning_rate=0.085537978248575, log_max_bin=9, 
                          colsample_bytree=0.7663773657187746, 
                          reg_alpha=0.006958608037974516, 
                          reg_lambda=0.4683303882185501)
svm = SVC(C=4.5, gamma=0.95, kernel='rbf')

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)

# Spacy-Vectorizer, leider nicht funktionsfähig

In [ ]:
# Ob diese Funktion funktionsfähig ist, wurde nicht getestet. Bei der Gird-SearchCV
# und Flaml ist die Funktion so, wie sie mal eines Abends funktioniert hat aber das 
# Programm nach 6h Laufzeit abgestützt ist.

def spacy_vectorizer(dataframe_lyrics, dataframe_label):
  '''
  input:
    dataframe_lyrics: dataframe mit den Liedtexten
    dataframe_label : dataframe mit den Genre

  output:
    result = array mit dem Liedtexte-Dataframe vektoisiert und mit den Genre
  '''
  # Funktion definieren, um die Liedtexte in Vektoren zu konvertieren
  def document_vector(text):
      doc = nlp(text)
      vectors = []
      for token in doc:
          if token.has_vector:
              vectors.append(token.vector)
      if len(vectors) > 0:
          return np.mean(vectors, axis=0)
      else:
          raise ValueError("Keine Vektoren gefunden")

  # Herausfinden, in welchen Spalten keine Vektoren vorhanden sind
  no_vector_docs = []
  for i, text in enumerate(dataframe_lyrics):
      try:
          vec = document_vector(text)
      except ValueError as e:
          print(f"Fehler: {e} bei Dokument {i}")
          no_vector_docs.append(i)
  print(f"Dokumente ohne Vektoren: {no_vector_docs}")

  # Alle Zeilen ohne Vektoren löschen
  for i in no_vector_docs:
    dataframe_lyrics = dataframe_lyrics.drop(i)
    dataframe_label = dataframe_label.drop(i)

  # Das Dataframe für die Labels reindexen
  dataframe_label = dataframe_label.reset_index(drop=True)  

  # Compute the document vectors
  documents_with_vectors = []
  labels_with_vectors = []
  for i in range(len(dataframe_lyrics)):
      text = dataframe_lyrics[i]
      dataframe_label = dataframe_label[i]
      try:
          vec = document_vector(text)
          documents_with_vectors.append(text)
          labels_with_vectors.append(dataframe_label)
      except ValueError as e:
          print(f"Fehler: {e}")

  X = [document_vector(text) for text in documents_with_vectors]
  y = labels_with_vectors
  result = [X,y]

In [ ]:
# Laden Sie die Textdokumente und die Klassenlabels
df_train_lyrics = df['cleaned_lyrics'] # Liste von Textdokumenten
df_train_labels = df['Genre'] # Liste von Klassenlabels (eins pro Dokument)

In [ ]:
X = spacy_vectorizer(df_test_lyrics, df_test_labels)
X_test = X[0]
y_test = X[1]
X_test = X_test.astype(np.float32)

In [ ]:
X = spacy_vectorizer(df_train_lyrics, df_train_labels)
X_train = X[0]
y_train = X[1]
X_train = X_train.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier()
svm = SVC()

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)